# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [10]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
llm_model = "deepseek-ai/DeepSeek-V3"

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_openai import OpenAI

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(
    file_path=file,
    encoding="utf-8"
)

In [18]:
from langchain.indexes import VectorstoreIndexCreator

In [19]:
os.environ["HF_ENDPOINT"]

'https://hf-mirror.com'

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings

# 初始化 Hugging Face 嵌入模型
embedding = HuggingFaceEmbeddings()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding
).from_loaders([loader])

e:\code\LangChain-for-LLM-Application-Development\venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [21]:
index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x0000000050D863C0>)

In [22]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [27]:
llm_replacement_model = OpenAI(temperature=0, model=llm_model)

In [28]:
response = index.query(query, llm=llm_replacement_model)

In [29]:
display(Markdown(response))

 Here is a table summarizing all the shirts with sun protection:

| Name                        | Gender | Sleeve Type | Fabric Composition | Sun Protection | Care Instructions | Key Features                                                                 |
|-----------------------------|--------|-------------|---------------------|----------------|-------------------|------------------------------------------------------------------------------|
| Women's Tropical Tee, Sleeveless | Women  | Sleeveless  | 71% nylon, 29% polyester (shell); 100% polyester (cape lining) | UPF 50+        | Machine wash and dry | Slightly fitted, wrinkle resistant, low-profile pockets, front and back cape venting, two front pockets, tool tabs, eyewear loop |
| Sun Shield Shirt by         | Unspecified | Unspecified | 78% nylon, 22% Lycra Xtra Life fiber | UPF 50+        | Handwash, line dry | Moisture-wicking, abrasion resistant, fits over swimsuit, recommended by The Skin Cancer Foundation |
| Men's Plaid Tropic Shirt, Short-Sleeve | Men    | Short-Sleeve | 52% polyester, 48% nylon | UPF 50+        | Machine wash and dry | Wrinkle-free, quick

## Step By Step

In [30]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(
    file_path=file,
    encoding="utf-8"
)

In [31]:
docs = loader.load()

In [32]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

In [34]:
embed = embedding.embed_query("Hi my name is Harrison")

In [35]:
print(len(embed))

768


In [36]:
print(embed[:5])

[0.03336680307984352, -0.004217864945530891, 0.005565308034420013, -0.0070279971696436405, 0.010666961781680584]


In [37]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embedding
)

In [38]:
query = "Please suggest a shirt with sunblocking"

In [39]:
docs = db.similarity_search(query)

In [40]:
len(docs)

4

In [41]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [42]:
retriever = db.as_retriever()

In [44]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [45]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [46]:
print(qdocs)

: 255
name: Sun Shield Shirt by
description: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. 

Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.

Fabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.

Additional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.

Sun Protection That Won't Wear Off
Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.: 679
name: Women's Tropical Tee, Sleeveless
description: Our five-star sleeveless button-up shirt has a fit to flatter and SunSmart™ protection to block the sun’s harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fa

In [58]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one."
) 

In [60]:
# display(Markdown(str(response.content)))
display(Markdown(response))

Here’s a table summarizing all the shirts with sun protection in Markdown format:

```markdown
| **Name**                     | **Description**                                                                                                                                                                                                 | **Fabric & Care**                                                                 | **Additional Features**                                                                                                                                                                                                 |
|------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Sun Shield Shirt**         | High-performance sun shirt with UPF 50+ protection, moisture-wicking, and abrasion-resistant fabric.                                                                                                           | 78% nylon, 22% Lycra Xtra Life. Handwash, line dry.                               | Wicks moisture, fits over swimsuits, abrasion-resistant, recommended by The Skin Cancer Foundation.                                                                                                                     |
| **Women's Tropical Tee**     | Sleeveless button-up shirt with SunSmart™ UPF 50+ protection, wrinkle-resistant, and flattering fit.                                                                                                           | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry. | Smoother buttons, low-profile pockets, side shaping, front and back cape venting, two front pockets, tool tabs, eyewear loop.                                                                                          |
| **Girls' Beachside Breeze**  | Half-sleeve swim shirt with UPF 50+ protection, snag- and fade-resistant fabric, and durable seawater-resistant material.                                                                                      | 80% nylon, 20% Lycra Xtra Life. Machine wash, line dry.                           | Rash guard, durable fabric, cover-stitched seams, recommended by The Skin Cancer Foundation.                                                                                                                           |
| **Sunrise Tee**              | Lightweight UV-protective button-down shirt with moisture-wicking, wrinkle-free fabric, and UPF 50+ protection.                                                                                                | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry. | Smoother buttons, low-profile pockets, side shaping, front and back cape venting, two front pockets, tool tabs, eyewear loop, wrinkle-free.                                                                             |
```

### Summary:
1. **Sun Shield Shirt**: A high-performance sun shirt with UPF 50+ protection, designed for moisture-wicking and durability. Ideal for outdoor activities and swimwear coverage.
2. **Women's Tropical Tee**: A sleeveless button-up shirt with SunSmart™ UPF 50+ protection, wrinkle resistance, and a flattering fit. Perfect for casual or active wear.
3. **Girls' Beachside Breeze**: A half-sleeve swim shirt with UPF 50+ protection, durable fabric, and seawater resistance. Great for beach or watersport activities.
4. **Sunrise Tee**: A lightweight, UV-protective button-down shirt with moisture-wicking, wrinkle-free fabric, and UPF 50+ protection. Suitable for hot weather and travel.

In [61]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [62]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [65]:
response = qa_stuff.invoke(query)
# response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [71]:
display(Markdown(response["result"]))
# display(response["result"])

Here’s a table summarizing all the shirts with sun protection:

| **Product ID** | **Name**                     | **Summary**                                                                                                                                                                                                 |
|----------------|------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 679            | Women's Tropical Tee, Sleeveless | Sleeveless button-up shirt with SunSmart™ UPF 50+ protection. Softly shapes the body, wrinkle-resistant, and features front and back cape venting, two front pockets, tool tabs, and an eyewear loop.        |
| 709            | Sunrise Tee                  | Lightweight, UV-protective button-down shirt with UPF 50+ protection. Moisture-wicking, wrinkle-free, and features smoother buttons, low-profile pockets, and front and back cape venting.                  |
| 255            | Sun Shield Shirt             | High-performance sun shirt with UPF 50+ protection. Moisture-wicking, abrasion-resistant, and fits comfortably over swimsuits. Recommended by The Skin Cancer Foundation for UV protection.                |
| 619            | Tropical Breeze Shirt        | Lightweight, breathable long-sleeve men’s shirt with SunSmart™ UPF 50+ protection. Wrinkle-resistant, moisture-wicking, and features front and back cape venting for added comfort.                         |

All shirts offer UPF 50+ sun protection, blocking 98% of the sun's harmful UV rays.

In [72]:
response = index.query(query, llm=llm)

In [73]:
display(Markdown(response))

Here is a table summarizing all the shirts with sun protection:

| **ID** | **Name**                          | **Summary**                                                                                                                                                                                                 |
|--------|-----------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 679    | Women's Tropical Tee, Sleeveless  | Sleeveless button-up shirt with SunSmart™ UPF 50+ protection. Made of 71% nylon, 29% polyester. Features wrinkle resistance, front and back cape venting, two front pockets, and machine washable.           |
| 255    | Sun Shield Shirt                  | High-performance sun shirt with UPF 50+ protection. Made of 78% nylon, 22% Lycra Xtra Life fiber. Moisture-wicking, abrasion-resistant, and handwashable. Recommended by The Skin Cancer Foundation.         |
| 374    | Men's Plaid Tropic Shirt, Short-Sleeve | Short-sleeve shirt with UPF 50+ protection. Made of 52% polyester, 48% nylon. Features wrinkle-free fabric, front and back cape venting, two front bellows pockets, and machine washable.                    |
| 619    | Tropical Breeze Shirt             | Long-sleeve men’s shirt with SunSmart™ UPF 50+ protection. Made of 71% nylon, 29% polyester. Features wrinkle resistance, moisture-wicking fabric, polyester-mesh inserts, and machine washable.             |

All shirts offer UPF 50+ protection, blocking 98% of the sun's harmful UV rays.